<a href="https://colab.research.google.com/github/d-tomas/data-mining/blob/main/notebooks/data_mining_4.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción de características

## Pasos previos

In [ ]:
# Importamos las librerías de Python que necesitaremos en este notebook

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pd.options.mode.chained_assignment = None  # Evitamos warnings indeseados

Vamos a trabajar de nuevo con dos conjuntos de datos en formato CSV

* `pokemon.csv`: contiene 41 características de cada uno de los 802 Pokemon desde la generación 1 hasta la 7
* `ansur_ii.csv`: *Anthropometric Survey of US Army Personnel* contiene 93 medidas corporales realizadas a 6.068 adultos (4.082 hombres y 1.986 mujeres)


In [ ]:
# Obtención de los ficheros CSV con los datos

!wget https://raw.githubusercontent.com/d-tomas/data-mining/main/datasets/pokemon.csv
!wget https://raw.githubusercontent.com/d-tomas/data-mining/main/datasets/ansur_ii.csv

In [ ]:
# Cargamos los datos de Pokemon en formato CSV

data_pokemon = pd.read_csv('pokemon.csv')
data_pokemon

In [ ]:
# Cargamos los datos de ANSUR II en formato CSV

data_ansur = pd.read_csv('ansur_ii.csv')
data_ansur

## Extracción manual

In [ ]:
# Cuando se conce bien un dataset, se pueden combinar variables antiguas en otras nuevas
# En el dataset ANSUR II se puede utilizar el índice de masa corporal para combinar peso y estatura
# Body Mass Index (BMI) = peso (kg) / estatura^2 (m^2)

data_body = pd.DataFrame(data_ansur['weightkg'] / 10)  # El peso está en hectogramos (??). Lo pasamos a kilogramos
data_body['stature'] = data_ansur['stature'] / 1000  # La estatura está en milímetros. La pasamos a metros
data_body['BMI'] = data_body['weightkg'] / data_body['stature'] ** 2
data_body

In [ ]:
# Si añadimos esta información al dataset, los valores de altura y peso pueden quedar obsoletos y los podemos eliminar

data_body.drop(['weightkg', 'stature'], axis=1, inplace=True)
data_body

 ## Análisis de componentes principales

In [ ]:
# Vamos a analizar primero qué relación hay entre el tamaño de las manos y el tamaño de los pies

plt.figure(figsize=(6, 6))
sns.scatterplot(data=data_ansur, x='handlength', y='footlength')
plt.show()

In [ ]:
# Para poder aplicar PCA hay que escalar las características primero
# El método 'StandardScaler' centra y escala (media 0 y varianza 1) cada característica de manera independiente

scaler = StandardScaler()
data_scaler = pd.DataFrame(scaler.fit_transform(data_ansur[['handlength', 'footlength']]), columns=data_ansur[['handlength', 'footlength']].columns)
plt.figure(figsize=(6, 6))
sns.scatterplot(data=data_scaler, x='handlength', y='footlength')
plt.show()

In [ ]:
# Podemos ver cómo ha quedado la media y la desviación tras el escalado

print('handlength')
print('Media: %f' % data_scaler['handlength'].mean())
print('Desviación: %f' % data_scaler['handlength'].std())
print('footlength')
print('Media: %f' % data_scaler['footlength'].mean())
print('Desviación: %f' % data_scaler['footlength'].std())

In [ ]:
# Entre las dos variables hay una fuerte correlación (el escalado no afecta)

data_scaler.corr()

In [ ]:
# Usamos 'PCA' y 'fit_transform' sobre los datos escalados para calcular los componentes principales

pca = PCA()
data_pca = pca.fit_transform(data_scaler)
pd.DataFrame(data_pca).corr()

In [ ]:
# Al mostrar el conjunto resultante, se ve que los datos ya no guardan correlación

plt.figure(figsize=(6, 6))
sns.scatterplot(x=data_pca[:, 0], y=data_pca[:, 1])
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
# Podemos usar el atributo 'explained_variance_ratio_' como métrica para evaluar la utilidad de los componentes principales
# Indica el porcentaje de varianza que se atribuye a cada uno de los componentes seleccionados
# El primer componente explica el 92% de la varianza en los datos y el segundo el resto

pca.explained_variance_ratio_

In [ ]:
# Un inconveniente de PCA es que los componentes con los que te quedas son difíciles de interpretar
# Para entender mejor los componentes se puede mirar el atributo 'components_'
# Indica hasta qué punto cada componente se ve afectado por una característica específica
# La característica que tenga el mayor efecto positivo o negativo sobre el componente puede usarse para darle significado
# En el primer componente [0.71, 0.71] el efecto de ambas características es el mismo
# El primer componente se ve tanto afectado por el tamaño de los pies como por el de las manos
# El segundo componente [0.71, -0.71] se ve negativamente afectado por el tamaño de los pies
# La gente que tiene una puntuación alta para el segundo componente tiene los pies pequeños comparados con sus manos

pca.components_

In [ ]:
# Vamos a ver otro ejemplo con un conjunto mayor de características altamente correlacionadas
# Cogemos todo el conjunto de datos de ANSUR II

scaler = StandardScaler()
data_num = data_ansur.select_dtypes(exclude='object')  # Nos quedamos con las columnas numéricas
data_scaler = pd.DataFrame(scaler.fit_transform(data_num), columns=data_num.columns)
pca = PCA()
pca.fit(data_scaler)
pca.explained_variance_ratio_  # Los dos primeros componentes explican 54,17% y 12,09% de la varianza

In [ ]:
# Podemos usar 'cumsum' para ver cuánta varianza podemos explicar en total usando un cierto número de componentes
# En este ejemplo, los dos primeros componentes nos permiten explicar el 66,27% de la varianza en los datos
# Con los diez primeros componentes podemos mantener el 82,87% de la varianza

pca.explained_variance_ratio_.cumsum()

In [ ]:
# Como PCA implica un escalado previo, se pueden combinar ambas operaciones con un pipeline

pipe = Pipeline([('scaler', StandardScaler()), ('reducer', PCA())])
pc = pipe.fit_transform(data_num)
pc[:, :2]  # Dos primeros componentes principales

In [ ]:
# El dataset de ANSUR II tiene una serie de columnas categoriales
# Podemos ver si estas características están alineadas con las fuentes más importantes de varianza en los datos

categories = data_ansur.select_dtypes(include='object')
categories.head()

In [ ]:
# Vamos a ver cómo está relacionado el género con la varianza
# Tomamos los dos primeros componentes principales

categories['PC1'] = pc[:, 0]
categories['PC2'] = pc[:, 1]
sns.scatterplot(data=categories, x='PC1', y='PC2', hue='Gender', alpha=0.4)
plt.show()  # Se aprecia más efecto en la primera componente principal

In [ ]:
# Podemos decirle a PCA cuántos componentes queremos que calcule con el atributo 'n_components'
# También podemos decirle la proporción mínima de varianza que queremos mantener

data_num = data_pokemon.select_dtypes(exclude='object')
data_num.dropna(inplace=True)  # Debemos eliminar NaN
pipe = Pipeline([('scaler', StandardScaler()), ('reducer', PCA(n_components=0.9))])  # Mantenemos el 90% de la varianza
pipe.fit(data_num)
len(pipe.steps[1][1].components_)

In [ ]:
# Podemos visualizar donde se concentra la mayoría de varianza explicada de los componentes principales
# El punto donde se da el cambio brusco en la gráfica se conoce como codo (elbow)
# Es un buen punto de partida para decidir el número de componentes a mantener

var = pipe.steps[1][1].explained_variance_ratio_
sns.lineplot(data=var)
plt.xlabel('Principal component index')
plt.ylabel('Explained variance ratio')
plt.show()  # Podríamos mantener 5 componentes en este caso

# Referencias

* [The Complete Pokemon Dataset](https://www.kaggle.com/rounakbanik/pokemon)
* [ANSUR II](https://www.openlab.psu.edu/ansur2/)
* [Características de ANSUR II explicadas](http://tools.openlab.psu.edu/publicData/ANSURII-MFR.pdf)